In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import nltk
import torch.optim as optim
from nltk.tokenize import word_tokenize,sent_tokenize

In [3]:
sent='''Good morning respected teachers and my dear friends,
Today, I would like to speak about an exciting and rapidly growing topic — Artificial Intelligence in Computer Vision.
Computer vision is a field of artificial intelligence that enables machines to see, understand, and interpret images and videos, just like humans do. While our eyes capture visual information, it is our brain that understands it. In the same way, computer vision uses AI algorithms and deep learning models to analyze visual data and make decisions.
One of the most important technologies behind computer vision is deep learning, especially Convolutional Neural Networks, also known as CNNs. These models learn patterns such as edges, shapes, colors, and objects from millions of images. As a result, machines can now recognize faces, read handwriting, detect objects, and even understand emotions from facial expressions.
AI-powered computer vision is already transforming many industries. In healthcare, it helps doctors detect diseases like cancer by analyzing X-rays and MRI scans more accurately and quickly. In transportation, computer vision plays a crucial role in self-driving cars, allowing vehicles to detect pedestrians, traffic signs, and road conditions in real time. In security, facial recognition systems are used for surveillance and identity verification. Even in agriculture, computer vision helps farmers monitor crop health and detect pests early.
However, along with its benefits, computer vision also raises important concerns. Issues such as data privacy, surveillance misuse, and algorithmic bias must be handled responsibly. It is essential that AI systems are designed ethically and used for the benefit of society.
In conclusion, artificial intelligence in computer vision is changing the way machines interact with the world. From saving lives to improving daily convenience, its impact is powerful and far-reaching. As students and future professionals, it is our responsibility to learn, innovate, and use this technology wisely.'''

In [4]:
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
tokens=word_tokenize(sent.lower())

In [9]:
sentence=sent_tokenize(sent.lower())

In [10]:
sentence

['good morning respected teachers and my dear friends,\ntoday, i would like to speak about an exciting and rapidly growing topic — artificial intelligence in computer vision.',
 'computer vision is a field of artificial intelligence that enables machines to see, understand, and interpret images and videos, just like humans do.',
 'while our eyes capture visual information, it is our brain that understands it.',
 'in the same way, computer vision uses ai algorithms and deep learning models to analyze visual data and make decisions.',
 'one of the most important technologies behind computer vision is deep learning, especially convolutional neural networks, also known as cnns.',
 'these models learn patterns such as edges, shapes, colors, and objects from millions of images.',
 'as a result, machines can now recognize faces, read handwriting, detect objects, and even understand emotions from facial expressions.',
 'ai-powered computer vision is already transforming many industries.',
 'in

In [11]:
vocab={"<unk>":0}
for token in tokens:
    if token not in vocab:
        vocab[token]=len(vocab)

In [13]:
len(vocab)

192

In [14]:
def text_to_numerical(sentes,vocab):
    numerical_sentence=[]
    for token in sentes:
        if token in vocab:
            numerical_sentence.append(vocab[token])
        else:
            numerical_sentence.append(vocab["<unk>"])
    return numerical_sentence

In [23]:
input_sequence_token=[]
for send in sentence:
    input_sequence_token.append(text_to_numerical(word_tokenize(send),vocab))

In [25]:
len(input_sequence_token)

18

In [27]:
training_sequence=[]
for sentences in input_sequence_token:
     for i in range(1,len(sentences)):
        training_sequence.append(sentences[:i+1])

In [29]:
len(training_sequence)

333

In [31]:
len_list=[]
for train in training_sequence:
    len_list.append(len(train))

In [32]:
max(len_list)

30

In [33]:
padding_=[]
for sequence in training_sequence:
    padding_.append([0]*(max(len_list)-len(sequence))+sequence)

In [36]:
x_torch=torch.tensor(padding_,dtype=torch.long)

In [37]:
x_torch

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   2,   3,   4],
        ...,
        [  0,   0,   0,  ..., 188, 189, 190],
        [  0,   0,   0,  ..., 189, 190, 191],
        [  0,   0,   0,  ..., 190, 191,  28]])

In [38]:
X=x_torch[:,:-1]
y=x_torch[:,-1]

In [42]:
class CustomDataset(Dataset):
    def __init__(self,X,y):
        self.X=X
        self.y=y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self,index):
        return self.X[index],self.y[index]

In [43]:
dataset=CustomDataset(X,y)

In [46]:
loader=DataLoader(dataset,batch_size=32,shuffle=True)

In [47]:
class Lstm(nn.Module):
    def __init__(self,vocal_size):
        super().__init__()
        self.embedding=nn.Embedding(vocal_size,100)
        self.lstm=nn.LSTM(100,150,batch_first=True)
        self.fc=nn.Linear(150,vocal_size)
    def forward(self,x):
        embedded=self.embedding(x)
        inter,(final,cell)=self.lstm(embedded)
        ouput=self.fc(final.squeeze(0))
        return ouput

In [ ]:
model=Lstm(len(vocab))
optimizer=optim.Adam(model.parameters(),lr=0.001)
epochs=60
criterion=nn.CrossEntropyLoss()
for epoch in range(epochs):
    total_loss=0
    for x_,y_ in loader:
        out=model(x_)
        loss=criterion(out,y_)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    print(total_loss)

In [65]:
input_text="I would like , it"
def prediction(model,vocab,input_text):
    token_=word_tokenize(input_text)
    numerical_text=text_to_numerical(token_,vocab)
    padded_=torch.tensor([0]*(max(len_list)-len(numerical_text))+numerical_text,dtype=torch.long).unsqueeze(0)
    output=model(padded_)
    value,index=torch.max(output,dim=1)
    return input_text+ " "+list(vocab.keys())[index]

In [66]:
prediction(model,vocab,input_text)

'I would like , it is'

In [68]:
import time
num_tokens=10
input_text_="One of the most important"
for i in range(num_tokens):
    output_text=prediction(model,vocab,input_text_)
    print(output_text)
    input_text_=output_text
    time.sleep(0.5)

One of the most important technologies
One of the most important technologies behind
One of the most important technologies behind computer
One of the most important technologies behind computer vision
One of the most important technologies behind computer vision is
One of the most important technologies behind computer vision is deep
One of the most important technologies behind computer vision is deep learning
One of the most important technologies behind computer vision is deep learning ,
One of the most important technologies behind computer vision is deep learning , especially
One of the most important technologies behind computer vision is deep learning , especially convolutional


In [69]:
# Function to calculate accuracy
def calculate_accuracy(model, loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in loader:
            # batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, loader)
print(f"Model Accuracy: {accuracy:.2f}%")

Model Accuracy: 98.50%
